In [50]:
import os
import io
import gzip

import pandas
import requests

In [15]:
url_base = 'http://www.drugbank.ca/system/downloads/current/'
filename = 'all_target_ids_all.csv.zip'
! wget --timestamping --directory-prefix download $url_base/$filename
! unzip -o download/$filename -d download
! rm download/$filename

--2015-05-08 12:23:41--  http://www.drugbank.ca/system/downloads/current//all_target_ids_all.csv.zip
Resolving www.drugbank.ca (www.drugbank.ca)... 162.243.242.197
Connecting to www.drugbank.ca (www.drugbank.ca)|162.243.242.197|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207452 (203K) [application/zip]
Saving to: ‘download/all_target_ids_all.csv.zip’

100%[======================================>] 207,452      394KB/s   in 0.5s   

2015-05-08 12:23:42 (394 KB/s) - ‘download/all_target_ids_all.csv.zip’ saved [207452/207452]

Archive:  download/all_target_ids_all.csv.zip
  inflating: download/all_target_ids_all.csv  


In [35]:
path = os.path.join('download', 'all_target_ids_all.csv')
target_df = pandas.read_csv(path)
target_df.head()

,ID,Name,Gene Name,GenBank Protein ID,GenBank Gene ID,UniProt ID,Uniprot Title,PDB ID,GeneCard ID,GenAtlas ID,HGNC ID,Species,Drug IDs
0,P45059,Peptidoglycan synthase FtsI,ftsI,1574687,L42023,P45059,FTSI_HAEIN,NaN,NaN,NaN,NaN,Haemophilus influenzae (strain ATCC 51907 / DS...,DB00303
1,P19113,Histidine decarboxylase,HDC,32109,X54297,P19113,DCHS_HUMAN,NaN,HDC,HDC,HGNC:4855,Human,DB00114; DB00117
2,Q9UI32,"Glutaminase liver isoform, mitochondrial",GLS2,6650606,AF110330,Q9UI32,GLSL_HUMAN,NaN,GLS2,GLS2,HGNC:29570,Human,DB00142
3,P00488,Coagulation factor XIII A chain,F13A1,182309,M22001,P00488,F13A_HUMAN,NaN,F13A1,F13A1,HGNC:3531,Human,DB01839; DB02340
4,P35228,"Nitric oxide synthase, inducible",NOS2,292242,L09210,P35228,NOS2_HUMAN,NaN,NOS2A,NOS2A,HGNC:7873,Human,DB00125; DB00155; DB01110; DB01234; DB01686; D...


In [45]:
pairs = list()

for i, row in target_df.iterrows():
    drugs = row['Drug IDs']
    if not drugs:
        continue
    drugs = drugs.split('; ')
    
    base = dict()
    base['uniprot_id'] = row['UniProt ID']
    base['uniprot_name'] = row['Uniprot Title']
    base['species']  = row['Species']
    base['hgnc_id'] = row['HGNC ID']
    
    for drug in drugs:
        pair = base.copy()
        pair['drugbank_id'] = drug
        pairs.append(pair)

pair_df = pandas.DataFrame.from_dict(pairs).
pair_df.head()

,drugbank_id,hgnc_id,species,uniprot_id,uniprot_name
0,DB00303,NaN,Haemophilus influenzae (strain ATCC 51907 / DS...,P45059,FTSI_HAEIN
1,DB00114,HGNC:4855,Human,P19113,DCHS_HUMAN
2,DB00117,HGNC:4855,Human,P19113,DCHS_HUMAN
3,DB00142,HGNC:29570,Human,Q9UI32,GLSL_HUMAN
4,DB01839,HGNC:3531,Human,P00488,F13A_HUMAN


In [53]:
response = requests.get('http://git.dhimmel.com/uniprot/data/map/GeneID.tsv.gz', stream=True)
text = io.TextIOWrapper(gzip.GzipFile(fileobj=response.raw))
uniprot_df = pandas.read_table(text, engine='python')
uniprot_df.rename(columns={'uniprot': 'uniprot_id', 'GeneID': 'entrez_gene_id'}, inplace=True)

In [55]:
pair_df = pair_df.merge(uniprot_df, how='left')
pair_df.head()

,drugbank_id,hgnc_id,species,uniprot_id,uniprot_name,entrez_gene_id
0,DB00303,NaN,Haemophilus influenzae (strain ATCC 51907 / DS...,P45059,FTSI_HAEIN,950626
1,DB00114,HGNC:4855,Human,P19113,DCHS_HUMAN,3067
2,DB00117,HGNC:4855,Human,P19113,DCHS_HUMAN,3067
3,DB00142,HGNC:29570,Human,Q9UI32,GLSL_HUMAN,27165
4,DB01839,HGNC:3531,Human,P00488,F13A_HUMAN,2162
